## D2 missing some routes [PR](https://github.com/cal-itp/data-analyses/issues/1582)

In [1]:
import _ct_district_data_prep
import _transit_routes_on_shn
# import _ct_district_visuals
import _report_utils
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS

In [2]:
import google.auth

credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [3]:
district = "02 - Redding"

In [4]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [5]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

In [6]:
portfolio_organization_names = list(operator_df.portfolio_organization_name.unique())

In [7]:
portfolio_organization_names

['Modoc Transportation Agency',
 'Siskiyou County',
 'Redding Area Bus Authority',
 'Tehama County',
 'Trinity County',
 'Plumas Transit Systems',
 'Lassen Transit Service Agency']

In [8]:
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

In [9]:
operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    storage_options={"token": credentials.token},
)[["portfolio_organization_name", "service_date", "recent_combined_name", "geometry"]]

operator_route_gdf = operator_route_gdf.loc[
    operator_route_gdf.portfolio_organization_name.isin(portfolio_organization_names)
]

In [10]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

In [11]:
# Save everything out for now
intersection_gdf = gpd.read_parquet(
        f"{GCS_FILE_PATH}transit_route_intersect_shn_50_gtfs_digest.parquet",
        storage_options={"token": credentials.token},
    )
    

In [12]:
open_data_portal_df = pd.read_parquet(
        f"{GCS_FILE_PATH}transit_route_shn_open_data_portal_50.parquet",
        storage_options={"token": credentials.token},
    )

In [13]:
open_data_portal_df.head(1)

,portfolio_organization_name,recent_combined_name,shn_route,district,pct_route_on_hwy_across_districts,on_shs
0,Alameda-Contra Costa Transit District,10 E. 14th St. - Mission,"238, 112, 185",4,39.0,Y


In [14]:
open_data_portal_df.loc[open_data_portal_df.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

,portfolio_organization_name,recent_combined_name,shn_route,district,pct_route_on_hwy_across_districts,on_shs
1636,Redding Area Bus Authority,Route 99X/Amtrak Thruway Route 3,"273, 99, 36, 44, 32, 5","2, 3",97.8,Y


In [16]:
open_data_portal_df.loc[open_data_portal_df.recent_combined_name.str.contains("389")]

,portfolio_organization_name,recent_combined_name,shn_route,district,pct_route_on_hwy_across_districts,on_shs
799,Eastern Contra Costa Transit Authority,389 Pittsburg-Bay Point BART / Bay Point,4,4,0.4,Y


In [15]:
open_data_df2 = open_data_portal_df.loc[
        (open_data_portal_df.pct_route_on_hwy_across_districts > 0)
        & (open_data_portal_df.District.str.contains(str(district_int)))
    ]

AttributeError: 'DataFrame' object has no attribute 'District'

In [ ]:
open_data_df3 = open_data_df2[open_data_df2['District'].apply(lambda x: str(district_int) in x.split(', '))]

In [ ]:
open_data_df3.District.unique()

In [ ]:
open_data_df3.loc[open_data_df3.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

In [ ]:
open_data_df2.District.unique()

In [ ]:
open_data_df2.loc[open_data_df2.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

In [ ]:
open_data_df2.loc[open_data_df2.recent_combined_name.str.contains("389")]

In [ ]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(0, str(district_int))
)
# district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [ ]:
transit_route_shs_gdf.info()

In [ ]:
transit_route_shs_table.loc[
    transit_route_shs_table["Portfolio Organization Name"]
    == "Redding Area Bus Authority"
].sort_values(by=["Route"])

In [ ]:
operator_route_gdf.loc[
    operator_route_gdf.portfolio_organization_name == "Redding Area Bus Authority"
].drop(columns=["geometry"]).sort_values(
    by=["service_date", "recent_combined_name"], ascending=[False, True]
).head(
    20
)

## Can't find Route 19 and Route 99X in the `_ct_district_data_prep.final_transit_route_shs_outputs` area

In [ ]:
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

subset = [
    "service_date",
    "geometry",
    "portfolio_organization_name",
    "recent_combined_name",
    "route_id",
]
op_geography_df = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    storage_options={"token": credentials.token},
)[subset]

In [ ]:
op_geography_df.portfolio_organization_name.unique()

In [ ]:
raba = op_geography_df.loc[
    op_geography_df.portfolio_organization_name == "Redding Area Bus Authority"
]

### Route 19 is missing but Route 99x is here.

In [ ]:
raba[["recent_combined_name", "route_id", "service_date"]].sort_values(
    by=["service_date", "recent_combined_name"], ascending=[False, True]
).drop_duplicates(subset=["recent_combined_name"])

In [ ]:
processed_transit_routes = _transit_routes_on_shn.process_transit_routes()

In [ ]:
processed_transit_routes.columns

In [ ]:
raba_only = processed_transit_routes.loc[
    processed_transit_routes.portfolio_organization_name == "Redding Area Bus Authority"
]

In [ ]:
raba_only.sort_values(
    by=["service_date", "recent_combined_name"], ascending=[False, True]
).drop(columns = ["geometry"])

In [ ]:
shn_raba = _transit_routes_on_shn.routes_shn_intersection(50,"ct_district_route")

In [ ]:
shn_raba  = shn_raba.loc[
    shn_raba.portfolio_organization_name == "Redding Area Bus Authority"
]

In [ ]:
shn_raba.columns

In [ ]:
shn_raba.shape

In [ ]:
route_99 = shn_raba.loc[shn_raba.recent_combined_name.str.contains("Route 99X/Amtrak Thruway Route 3")]

In [ ]:
route_99 = route_99.rename(columns={"pct_route_on_hwy": "pct_route_on_hwy_across_districts"})

In [ ]:
def group_route_district(df: pd.DataFrame, pct_route_on_hwy_agg: str) -> pd.DataFrame:

    # Aggregate by adding all the districts and SHN to a single row, rather than
    # multiple and sum up the total % of SHN a transit route intersects with
    agg1 = (
        df.groupby(
            [
                "portfolio_organization_name",
                "recent_combined_name",
            ],
            as_index=False,
        )[["shn_route", "district", "pct_route_on_hwy_across_districts"]]
        .agg(
            {
                "shn_route": lambda x: ", ".join(set(x.astype(str))),
                "district": lambda x: ", ".join(set(x.astype(str))),
                "pct_route_on_hwy_across_districts": pct_route_on_hwy_agg,
            }
        )
        .reset_index(drop=True)
    )

    # Clean up
    agg1.pct_route_on_hwy_across_districts = (
        agg1.pct_route_on_hwy_across_districts.astype(float).round(2)
    )
    return agg1

In [ ]:
agg1 = group_route_district(route_99, "sum")

In [ ]:
agg1


In [ ]:
agg1.loc[
        agg1.district.astype(str).str.contains(str(district_int))
    ]

In [ ]:
open_data_portal_df = _transit_routes_on_shn.prep_open_data_portal(raba_only)